In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def generate_text(prompt, max_length=5000, num_return_sequences=1):
    # Tokenize the input prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    
    # Generate text using the model
    outputs = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,  # Prevents repeating phrases
        do_sample=True,          # Enables sampling; set to False for greedy decoding
        top_k=50,                # Limits to the top 50 tokens for sampling
        top_p=0.95,              # Nucleus sampling: considers tokens with cumulative probability of 0.95
        temperature=0.7,         # Controls randomness in predictions
        early_stopping=True      # Stops when an end-of-sentence token is generated
    )
    
    # Decode the output tokens to strings
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return generated_texts

In [3]:
import json

with open("/share/u/harshraj/CotIF/src/synthesize/data/forward/cotroller_train_dataset-mix-with_translation.json", "r") as file:
    data = [json.loads(line) for line in file]

In [7]:
from pprint import pprint

pprint(data[2])

{'applied_functions': 'indent_paragraphs_forward',
 'messages': [{'content': 'Return your final response within \\boxed{}. When '
                          '$\\frac{3}{1250}$ is written as a decimal, how many '
                          'zeros are there between the decimal point and the '
                          'first non-zero digit?\n'
                          '\n'
                          '## NOTES\n'
                          '\n'
                          'Instructions that you must follow in your reasoning '
                          'chain (the part of your response before your final '
                          'answer):\n'
                          '- indent each paragraph with a tab character',
               'role': 'user'},
              {'content': '<think>\n'
                          'Okay, so before I start reasoning, let me recall '
                          "the user's instructions for my reasoning chain: I "
                          'must indent each paragraph wi

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Harsh1729/qwenr1_1.5b_lora_sft-cotif-v1")
model = AutoModelForCausalLM.from_pretrained("Harsh1729/qwenr1_1.5b_lora_sft-cotif-v1", device_map="auto")

In [ ]:
print(texts[0].replace(prompt, ""))

In [ ]:
prompt = "When generating your thought process between `<think>` and `</think>`, please ensure adherence to the following guidelines:\n\n1. Your explanation should consist of at least one but no more than three paragraphs.\n2. As we have identified excessive use of parentheses as a concern (`'contains_excessive_parentheses': True`), try to minimize their usage without compromising clarity.\n3. You are free to include letters containing the letter \"e\" if necessary (`'avoids_letter_e': False`).\n4. Refrain from incorporating emojis into your response (`'contains_emojis': False`).\n5. Since we need to track palindrome occurrences, keep note of palindromes found during your reasoning; however, this specific task does not require reaching the target count of 182 palindromes (`'palindrome_count': 182`). \n6. Organize your thoughts using lists or enumeration when helpful for understanding (`'contains_list_or_enumeration': True`)\n\nNow answer this question: A student takes a multiple choice test with $5$ questions. Each question has $4$ possible choices, and only $1$ of them is correct. Let $X$ be the random variable representing the number of correct answers the student gets. Then $X$ follows a binomial distribution with parameters $n=5$ and $p=\\frac{1}{4}$. Compute $P(X=4)$."
texts = generate_text(prompt).replace(prompt, "")
for i, text in enumerate(texts, 1):
    print(f"\nGenerated text {i}:\n{text}")


In [ ]:
import json 
data = []
with open("/share/u/harshraj/CotIF/data/data_1k-2.jsonl", "r") as file:
    for line in file:
        data.append(json.loads(line))

In [ ]:
print(data[0][0]['content'])

In [ ]:
print(data[0][1]['content'])

In [ ]:
import json

with open("/share/u/harshraj/CotIF/data/data_1k-using_templates-train-1.jsonl", "r") as file:
    data = [json.loads(line) for line in file]

In [ ]:
from pprint import pprint
pprint(data[100])

In [ ]:
%%capture
!pip install snac ipywebrtc
!pip install datasets

In [ ]:
from snac import SNAC
import torch
import torch
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
import numpy as np
import soundfile as sf
import IPython.display as ipd
import librosa
from ipywebrtc import AudioRecorder, Audio
from IPython.display import display
import ipywidgets as widgets
from huggingface_hub import snapshot_download
import torchaudio.transforms as T
import librosa
import torch
from IPython.display import Audio, display

model_name = "canopylabs/orpheus-tts-0.1-pretrained"
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
snac_model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz").eval().cuda()


# Download only model config and safetensors
model_path = snapshot_download(
    repo_id=model_name,
    allow_patterns=[
        "config.json",
        "*.safetensors",
        "model.safetensors.index.json",
    ],
    ignore_patterns=[
        "optimizer.pt",
        "pytorch_model.bin",
        "training_args.bin",
        "scheduler.pt",
        "tokenizer.json",
        "tokenizer_config.json",
        "special_tokens_map.json",
        "vocab.json",
        "merges.txt",
        "tokenizer.*"
    ]
)

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
model.cuda()

In [ ]:
### CHANGE THIS TO YOUR OWN FILE AND TEXT

my_wav_file_is = "lgt/voice_acting_outputs-sage-emotion-prompts_sample_1288.wav"
and_the_transcript_is = "Something or the other"

the_model_should_say = [
  "I finally got into the university of my dreams! I can't believe all this hard work actually  paid off!",
  "Why is your frickin' Waymo blocking the frickin' road? GET OUT OF THE WAY!",
  "I'm so sorry to hear about your pet, but you know, he'll pull through.",
  "Conversational, uhm, systems, tend to speak pretty robotically, because- because they don't, really understand how, uhm, humans talk."

]

In [ ]:
''' Here we tokenise the prompt you gave us, we also tokenise the prompts you want the model to say

The template is:

start_of_human, start_of_text, text, end_of_text, start_of_ai, start_of_speech, speech, end_of_speech, end_of_ai, start_of_human, text, end_of_human and then generate from here

'''


filename = my_wav_file_is

audio_array, sample_rate = librosa.load(filename, sr=24000)

def tokenise_audio(waveform):
  waveform = torch.from_numpy(waveform).unsqueeze(0)
  waveform = waveform.to(dtype=torch.float32)


  waveform = waveform.unsqueeze(0)

  with torch.inference_mode():
    codes = snac_model.encode(waveform)

  all_codes = []
  for i in range(codes[0].shape[1]):
    all_codes.append(codes[0][0][i].item()+128266)
    all_codes.append(codes[1][0][2*i].item()+128266+4096)
    all_codes.append(codes[2][0][4*i].item()+128266+(2*4096))
    all_codes.append(codes[2][0][(4*i)+1].item()+128266+(3*4096))
    all_codes.append(codes[1][0][(2*i)+1].item()+128266+(4*4096))
    all_codes.append(codes[2][0][(4*i)+2].item()+128266+(5*4096))
    all_codes.append(codes[2][0][(4*i)+3].item()+128266+(6*4096))


  return all_codes

myts = tokenise_audio(audio_array)
start_tokens = torch.tensor([[ 128259]], dtype=torch.int64)
end_tokens = torch.tensor([[128009, 128260, 128261, 128257]], dtype=torch.int64)
final_tokens = torch.tensor([[128258, 128262]], dtype=torch.int64)
voice_prompt = and_the_transcript_is
prompt_tokked = tokenizer(voice_prompt, return_tensors="pt")

input_ids = prompt_tokked["input_ids"]

zeroprompt_input_ids = torch.cat([start_tokens, input_ids, end_tokens, torch.tensor([myts]), final_tokens], dim=1) # SOH SOT Text EOT EOH

prompts = the_model_should_say

all_modified_input_ids = []
for prompt in prompts:
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  second_input_ids = torch.cat([zeroprompt_input_ids, start_tokens, input_ids, end_tokens], dim=1)
  all_modified_input_ids.append(second_input_ids)


all_padded_tensors = []
all_attention_masks = []

max_length = max([modified_input_ids.shape[1] for modified_input_ids in all_modified_input_ids])

for modified_input_ids in all_modified_input_ids:
  padding = max_length - modified_input_ids.shape[1]
  padded_tensor = torch.cat([torch.full((1, padding), 128263, dtype=torch.int64), modified_input_ids], dim=1)
  attention_mask = torch.cat([torch.zeros((1, padding), dtype=torch.int64), torch.ones((1, modified_input_ids.shape[1]), dtype=torch.int64)], dim=1)
  all_padded_tensors.append(padded_tensor)
  all_attention_masks.append(attention_mask)

all_padded_tensors = torch.cat(all_padded_tensors, dim=0)
all_attention_masks = torch.cat(all_attention_masks, dim=0)

input_ids = all_padded_tensors.to("cuda")
attention_mask = all_attention_masks.to("cuda")

In [ ]:
codes[1].shape[1]

In [ ]:
with torch.no_grad():
  generated_ids = model.generate(
      input_ids=input_ids,
      # attention_mask=attention_mask,
      max_new_tokens=990,
      do_sample=True,
      temperature=0.5,
      # top_k=40,
      top_p=0.9,
      repetition_penalty=1.1,
      num_return_sequences=1,
      eos_token_id=128258,
      # end_token_id=128009
  )

# generated_ids = torch.cat([generated_ids, torch.tensor([[128262]]).to("cuda")], dim=1) # EOAI

In [ ]:
token_to_find = 128257
token_to_remove = 128258

# Check if the token exists in the tensor
token_indices = (generated_ids == token_to_find).nonzero(as_tuple=True)

if len(token_indices[1]) > 0:
    last_occurrence_idx = token_indices[1][-1].item()
    cropped_tensor = generated_ids[:, last_occurrence_idx+1:]
else:
    cropped_tensor = generated_ids

mask = cropped_tensor != token_to_remove
processed_rows = []
for row in cropped_tensor:
    # Apply the mask to each row
    masked_row = row[row != token_to_remove]
    processed_rows.append(masked_row)

code_lists = []
for row in processed_rows:
    # row is a 1D tensor with its own length
    row_length = row.size(0)
    new_length = (row_length // 7) * 7  # largest multiple of 7 that fits in this row
    trimmed_row = row[:new_length]
    trimmed_row = [t - 128266 for t in trimmed_row]
    code_lists.append(trimmed_row)

def redistribute_codes(code_list):
  layer_1 = []
  layer_2 = []
  layer_3 = []
  for i in range((len(code_list)+1)//7):
    layer_1.append(code_list[7*i])
    layer_2.append(code_list[7*i+1]-4096)
    layer_3.append(code_list[7*i+2]-(2*4096))
    layer_3.append(code_list[7*i+3]-(3*4096))
    layer_2.append(code_list[7*i+4]-(4*4096))
    layer_3.append(code_list[7*i+5]-(5*4096))
    layer_3.append(code_list[7*i+6]-(6*4096))
  codes = [torch.tensor(layer_1).unsqueeze(0),
         torch.tensor(layer_2).unsqueeze(0),
         torch.tensor(layer_3).unsqueeze(0)]
  audio_hat = snac_model.decode(codes)
  return audio_hat

my_samples = []
for code_list in code_lists:
  samples = redistribute_codes(code_list)
  my_samples.append(samples)

In [ ]:
from IPython.display import Audio, display
for samples in my_samples:
  display(Audio(samples.detach().squeeze().to("cpu").numpy(), rate=24000))

In [ ]:
audio_array, sample_rate = librosa.load("/share/u/harshraj/CotIF/notebooks/lgt/voice_acting_outputs-sage-emotion-prompts_sample_1288.wav", sr=24000)

In [ ]:
def tokenise_audio(waveform):
  waveform = torch.from_numpy(waveform).unsqueeze(0)
  waveform = waveform.to(dtype=torch.float32)


  waveform = waveform.unsqueeze(0)

  with torch.inference_mode():
    codes = snac_model.encode(waveform)

  all_codes = []
  for i in range(codes[0].shape[1]):
    all_codes.append(codes[0][0][i].item()+128266)
    all_codes.append(codes[1][0][2*i].item()+128266+4096)
    all_codes.append(codes[2][0][4*i].item()+128266+(2*4096))
    all_codes.append(codes[2][0][(4*i)+1].item()+128266+(3*4096))
    all_codes.append(codes[1][0][(2*i)+1].item()+128266+(4*4096))
    all_codes.append(codes[2][0][(4*i)+2].item()+128266+(5*4096))
    all_codes.append(codes[2][0][(4*i)+3].item()+128266+(6*4096))


  return all_codes

In [ ]:
waveform = audio_array
waveform = torch.from_numpy(waveform).unsqueeze(0)
waveform = waveform.to(dtype=torch.float32)


waveform = waveform.unsqueeze(0)

with torch.inference_mode():
    codes = snac_model.encode(waveform)

all_codes = []
for i in range(codes[0].shape[1]):
    all_codes.append(codes[0][0][i].item()+128266)
    all_codes.append(codes[1][0][2*i].item()+128266+4096)
    all_codes.append(codes[2][0][4*i].item()+128266+(2*4096))
    all_codes.append(codes[2][0][(4*i)+1].item()+128266+(3*4096))
    all_codes.append(codes[1][0][(2*i)+1].item()+128266+(4*4096))
    all_codes.append(codes[2][0][(4*i)+2].item()+128266+(5*4096))
    all_codes.append(codes[2][0][(4*i)+3].item()+128266+(6*4096))

In [ ]:
codes[2].shape[1]

In [ ]:
(4*i)+3

In [ ]:
codes = snac_model.encode(waveform.cuda())

In [ ]:
myts = tokenise_audio(audio_array)
myts

In [ ]:
import re
from collections import Counter

from nltk.corpus import wordnet as wn
import spacy 
from string import punctuation

spacy_nlp = spacy.load('en_core_web_sm')

numbering_list = ['3', '7)', '7.', '4', 'iii.', 'iii-', '8.', '4-', 'v:', 'I:', 'ii.', 'i.', 'V)', 'E)', 'I)', 'III.', 'III)', '2-', '1)', 'v-', 'III', 'I.', 'c)', '1.', 'V-', 'iv)', 'A)', 'v)', 'IV', 'C.', 'ii)', 'I', 'IV.', 'C)', 'II-', '2.', 'III-', 'IV)', 'd)', 'iii', 'i-', 'iii:', 'A.', 'B.', '1', '6)', 'ii', '8)', '3)', 'e)', 'ii-', '5-', 'II)', 'iv-', '2)', 'e.', 'IV:', 'III:', 'i)', '10.', 'V', 'V.', 'v.', 'D)', 'E.', 'iv:', 'B)', 'II', 'ii:', 'V:', 'a.', '5.', 'IV-', '9.', 'D.', '3.', '4:', '2:', 'i', 'II.', '3-', '2', 'c.', 'a)', '3:', '10)', 'd.', 'i:', 'iv.', '1-', '4.', '5', 'iv', 'iii)', 'b.', '1:', 'II:', 'v', '5:', '6.', 'b)', 'I-', '9)', '4)', '5)']

stopwords_list = ['es', 'ing', 'ed', 'include', 'includes', 'also', 'haven', 'are', 'why', 'most', "won't", 'against', 'with', 'needn', 'couldn', 'now', 'mustn', 'who', 'under', 'doing', 'am', 'aren', 'they', "didn't", 'd', 'doesn', 'if', 'he', 'her', "haven't", 'isn', 'own', 'does', 'such', 'until', 'into', 'had', 'again', 'over', "hadn't", "you'll", 't', 'by', 'be', "wasn't", 'so', 'yours', 'both', 'any', 'did', "you've", 'these', 'myself', 'o', 'hasn', "isn't", 'you', 'other', 'shan', 'being', 'yourselves', 'was', 'no', 'm', 'those', 'will', 'its', 'itself', 'have', 'down', 'weren', 'having', 'wouldn', 'herself', "mustn't", 'very', 'do', "should've", 'him', "you'd", 'below', 'just', 'that', 'for', 'which', 'but', 'nor', 'all', 'then', 'i', 'whom', 'it', 'once', 'here', 've', "you're", 'ours', "that'll", 'a', 'won', 'himself', 'where', 'this', 'your', "hasn't", 'same', 'when', 'ourselves', 'because', "needn't", 'theirs', 'from', 'mightn', 'my', 'while', 'yourself', "she's", 'each', "doesn't", 'only', 'at', 's', 'their', "wouldn't", 'shouldn', 'and', 'themselves', 'hers', 'has', 'up', 'ma', 'in', 'll', 'we', 're', 'y', 'of', 'after', 'our', "shan't", 'before', 'wasn', 'can', 'should', 'been', 'through', 'as', 'further', 'during', 'between', 'there', 'me', 'on', 'don', "shouldn't", 'more', 'out', "don't", 'the', "weren't", "aren't", "it's", 'what', 'or', "couldn't", 'hadn', "mightn't", 'his', 'above', 'to', 'how', 'few', 'off', 'them', 'didn', 'ain', 'not', 'she', 'an', 'than', 'too', 'is', 'some', 'were', 'about']

common_title_words_set = {'introduction', 'conclusion', 'section', 'chapter', 'works', 'notes', 'note', 'further', 'see', 'references', 'reference', 'section', 'title', 'conclusion', 'intro', 'introduction', 'executive', 'summary', 'key', 'plot', 'theme'}
stopwords_set = set(stopwords_list + numbering_list)

def strip_left_stopwords(e_text):
  """
  Removes common stopwords from the left side of a text until a significant word is found.

  Args:
      e_text (str): The text to strip from the left side.

  Returns:
      str: Text with left-side stopwords removed.
  """
  e_text2 = []
  add_rest = False
  for et in e_text.split():
      if add_rest or ((et.lower() not in stopwords_set and et.lower() not in common_title_words_set) or et.lower().strip(".") in {"a", "an", "united", "the", "new", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten",  "asian", "american", "african", "european", }):
        add_rest = True
        e_text2.append(et)
  return " ".join(e_text2)

In [ ]:
def analyze_nouns_common_hypernyms(text):
    doc = spacy_nlp(text)
    # candidate_nouns = [strip_left_stopwords(e.text)  for e in doc.noun_chunks if len(e.text) > 4 and e.text.lower() not in stopwords_set]        
    candidate_nouns = [
        token.lemma_ for token in doc 
        if token.pos_ == "NOUN" and token.lemma_.lower() not in stopwords_set and len(token.text) > 2
    ]
    noun_synsets = {}
    for noun in candidate_nouns:
        syns = wn.synsets(noun, "n")
        if syns:
            noun_synsets[noun] = syns[0]

    hypernym_map = {}
    noun_list = sorted(noun_synsets.keys())
    for i in range(len(noun_list)):
        for j in range(i+1, len(noun_list)):
            syn_i = noun_synsets[noun_list[i]]
            syn_j = noun_synsets[noun_list[j]]
            lch = syn_i.lowest_common_hypernyms(syn_j)
            if lch:
                for common_hyp in lch:
                    name = common_hyp.lemmas()[0]
                    if name.name() in {"act", "whole", "communication", "thing", "content", "group", "social group", "unit", "organism", "region", "area", "geographic point", "relation", "attribute", "happening", "action", "entity", "causal agent", 'administrative district', "abstraction", "object", "point", "event", "physical entity", "matter", "part", "person", "state"}:
                        continue
                    hypernym_map.setdefault(name, set()).update([noun_list[i], noun_list[j]])
    
    hypernym_map = {
        hyp: list(nset) for hyp, nset in hypernym_map.items() if len(nset) >= 2
    }

    return hypernym_map

ss = """The scientist observed the dolphin and the octopus in the aquarium.  
Nearby, a photographer captured images of a lion and a zebra in the savanna exhibit.  
Visitors also saw a helicopter hovering over the mountains, and a truck delivering supplies to a remote village.  
Inside the exhibition hall, children interacted with a robot and explored a model of the solar system."""
obj = analyze_nouns_common_hypernyms(ss)

In [ ]:
ss = """The scientist observed the dolphin and the octopus in the aquarium.  
Nearby, a photographer captured images of a lion and a zebra in the savanna exhibit.  
Visitors also saw a helicopter hovering over the mountains, and a truck delivering supplies to a remote village.  
Inside the exhibition hall, children interacted with a robot and explored a model of the solar system."""

def generate_ngrams(tokens, n=4):
    return [tokens[i:i+n] for i in range(len(tokens)-n+1)]

def hidden_acrostic(text, length=5):
    first_letters = "".join([s.strip()[0].lower() for s in text.split('.') if s.strip()])
    for word in generate_ngrams(first_letters, 6):
        syns = wn.synsets(word)
        if syns:
            return word
        
    for word in generate_ngrams(first_letters, 5):
        syns = wn.synsets(word)
        if syns:
            return word
        
    for word in generate_ngrams(first_letters, 4):
        syns = wn.synsets(word)
        if syns:
            return word        

hidden_acrostic(ss)

In [ ]:
import random
k, v = random.sample(list(obj.items()), 1)[0]

In [ ]:
k

In [ ]:
v

In [ ]:
doc = spacy_nlp(ss)
candidate_nouns = [
    token.lemma_ for token in doc 
    if token.pos_ == "NOUN" and token.lemma_.lower() not in stopwords_set and len(token.text) > 2
]
candidate_nouns

In [ ]:


hypernym_map = {}
noun_list = sorted(noun_synsets.keys())
for i in range(len(noun_list)):
    for j in range(i+1, len(noun_list)):
        syn_i = noun_synsets[noun_list[i]]
        syn_j = noun_synsets[noun_list[j]]
        lch = syn_i.lowest_common_hypernyms(syn_j)
        if lch:
            for common_hyp in lch:
                name = common_hyp.lemmas()[0]
                if name in {"act", "whole", "communication", "thing", "content", "group", "social group", "unit", "organism", "region", "area", "geographic point", "relation", "attribute", "happening", "action", "entity", "causal agent", 'administrative district', "abstraction", "object", "point", "event", "physical entity", "matter", "part", "person", "state"}:
                    continue
                hypernym_map.setdefault(name, set()).update([noun_list[i], noun_list[j]])

hypernym_map = {
    hyp: list(nset) for hyp, nset in hypernym_map.items() if len(nset) >= 2
}

In [ ]:
%%capture
!pip install textblob
!python -m textblob.download_corpora

In [ ]:
import random
from textblob import TextBlob
import random
import re

def replace_words_with_emojis(text):
    emoji_map = {
        'error': '❌',
        'warning': '⚠️',
        'analysis': '🔍',
        'result': '✅',
        'success': '🎉',
        'failure': '💔',
        'logic': '🤔',
        'calculate': '🧮',
        'problem': '❓',
        'solution': '💡',
        'reasoning': '🧠',
        'trace': '📝',
        'thought': '💭',
        'data': '📊',
        'compute': '💻',
        'model': '🤖',
        'optimization': '⚙️',
        'test': '🧪',
        'experiment': '🔬',
        'input': '📥',
        'output': '📤',
        'unknown': '❓',
        'uncertain': '🤷',
        'prompt': '💬',
        'instruct': '📝'
    }
    
    pattern = r'\b(' + '|'.join(re.escape(word) for word in emoji_map.keys()) + r')\b'
    
    replaced = {}
    
    def replacer(match):
        original = match.group(0)
        lower_word = original.lower()
        emoji = emoji_map[lower_word]
        replaced.setdefault(original, emoji)
        return emoji
    
    if len(replaced.keys()) > 1:
        new_text = re.sub(pattern, replacer, text, flags=re.IGNORECASE)
        description = ("in your reasoning chain replace the words {} with their corresponding emojis {} respectively"
                    .format(list(replaced.keys()), list(replaced.values())))
    else:
        new_text, description, None, None
    return {'description': description, 'new_text': new_text}

# Example test text (simulating an AI reasoning trace)
sample_text = (
    "I performed an analysis of the data and computed the result. "
    "However, a warning was issued due to an error in the model. "
    "The reasoning was uncertain, but the solution eventually emerged from the test experiments."
)

print(replace_words_with_emojis(sample_text))


In [ ]:
text = "Okay, let's tackle this logic problem.\nThe question is asking which of the following is equivalent to \"If P is true, then Q is false.\" Hmm, first, I need to recall how to translate conditional statements into logical expressions.\nSo, the original statement is \"If P, then not Q.\" In symbolic terms, that would be P \u2192 \u00acQ, right?\nNow, the next step is to figure out the logical equivalents of this statement.\nI remember that implications can be converted using contrapositives or other equivalences.\nLet me think.\nThe contrapositive of P \u2192 \u00acQ would be Q \u2192 \u00acP because the contrapositive switches and negates the antecedent and consequent.\nSo, if the original is \"If P, then not Q,\" then the contrapositive is \"If Q, then not P.\" That seems right.\nAre those two equivalent?\nYes, in classical logic, an implication and its contrapositive are logically equivalent.\nSo that's one possible equivalent statement.\nBut the question is probably presenting multiple options, so maybe there are other forms.\nLet me recall other logical equivalences.\nFor example, an implication P \u2192 Q is equivalent to \u00acP \u2228 Q.\nSo applying that to P \u2192 \u00acQ, it would be equivalent to \u00acP \u2228 \u00acQ.\nWhich can also be written as \u00ac(P \u2227 Q) because by De Morgan's laws, \u00acP \u2228 \u00acQ is the same as \u00ac(P \u2227 Q).\nSo that's another way to express it: \"It is not the case that both P and Q are true.\" Hmm, so \"If P is true, then Q is false\" can also be phrased as \"P and Q cannot both be true at the same time.\" That makes sense.\nWait"
replace_words_with_emojis(text)

In [ ]:
import json 
with open("/share/u/harshraj/CotIF/data/cotroller_train_dataset-mix.json", "r") as file:
    data = [json.loads(line) for line in file]

In [ ]:
from pprint import pprint
pprint(data[0])

In [ ]:
from pprint import pprint
pprint(data[1])

In [ ]:
import spacy
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(vec1, vec2):
    """Return the cosine similarity between two vectors."""
    # Added a small constant to avoid division by zero.
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2) + 1e-8)

def check_for_metaphor(text, threshold=0.3):
    """
    Check for sentences that may contain metaphors based on low similarity between
    the subject and the complement of a linking verb.
    
    Parameters:
      text (str): The text to analyze.
      threshold (float): Similarity threshold below which a sentence is flagged as metaphorical.
      
    Returns:
      list: A list of sentences likely containing metaphors.
    """
    # Load spaCy's medium English model (ensure you have 'en_core_web_md' installed)
    nlp = spacy.load("en_core_web_md")
    doc = nlp(text)
    metaphor_sentences = []
    
    for sent in doc.sents:
        # Iterate over tokens in the sentence (no need to re-process the sentence)
        for token in sent:
            # Check if the token is a form of 'be' and is tagged as a VERB or AUX
            if token.lemma_ == "be" and token.pos_ in {"VERB", "AUX"}:
                # Find nominal subject and attribute/complement
                subj = [child for child in token.children if child.dep_ in ("nsubj", "nsubjpass")]
                comp = [child for child in token.children if child.dep_ in ("attr", "acomp")]
                if subj and comp:
                    sim = cosine_similarity(subj[0].vector, comp[0].vector)
                    # If the similarity is below the threshold, flag the sentence as metaphorical
                    if sim < threshold:
                        metaphor_sentences.append(sent.text)
                        break  # Proceed to the next sentence once a metaphor is found
    return metaphor_sentences

# Example usage:
if __name__ == "__main__":
    # sample_text = "Time is a thief. The sky is blue. He is as brave as a lion."
    metaphors = check_for_metaphor(sample_text)
    print("Sentences with potential metaphors:")
    for m in metaphors:
        print(" -", m)


In [ ]:
sample_text = """Okay, so I need to solve this age problem. Let me read it again carefully. The question says:\n\n\"Two years ago Pete was three times as old as his cousin Claire. Two years before that, Pete was four times as old as Claire. In how many years will the ratio of their ages be 2 : 1? The options are A through E: 2, 4, 5, 6, 8.\"\n\nHmm, age problems usually involve setting up equations based on the given information. Let me try to break this down step by step.\n\nFirst, let's define variables for their current ages. Let me denote Pete's current age as P and Claire's current age as C. The goal is to find how many years into the future their ages will have a ratio of 2:1. Let's call that number of years x. So, in x years, Pete will be P + x years old, and Claire will be C + x years old, and we need (P + x)\/(C + x) = 2\/1, which simplifies to P + x = 2(C + x). That equation might come in handy later. Let's keep that in mind.\n\nNow, the problem gives information about their ages in the past. Two"""

In [ ]:
ss = data[0]['messages'][1]['content'] + data[0]['messages'][2]['content']

In [ ]:
to_delete = "{% if '</think>' in content %}{% set content = content.split('</think>')[-1] %}{% endif %}"
new_template = tokenizer.get_chat_template().replace(to_delete, "")
tokenizer = AutoTokenizer.from_pretrained(MODEL, chat_template=new_template)
# tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
pprint(tokenizer.get_chat_template())

In [ ]:
pprint(data[0]['messages'])

In [ ]:
new_data_point = [data[0]['messages'][0]] + [data[0]['messages'][2]]

In [ ]:
pprint(new_data_point)

In [ ]:
new_data_point[1]['content'] = '<think>' + new_data_point[0]['content']

In [ ]:
pprint(tokenizer.apply_chat_template(new_data_point, tokenize=False))

In [ ]:
pprint(tokenizer.apply_chat_template(data[0]['messages'], tokenize=False))

In [ ]:
pprint(ss)

In [ ]:
pprint(tokenizer.get_chat_template())

In [ ]:
from transformers import AutoTokenizer
MODEL = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

def get_tokenizer_with_new_chat_template(tokenizer):
    to_delete = "{% if '</think>' in content %}{% set content = content.split('</think>')[-1] %}{% endif %}"
    new_template = tokenizer.get_chat_template().replace(to_delete, "")
    return AutoTokenizer.from_pretrained(MODEL, chat_template=new_template)

tokenizer = get_tokenizer_with_new_chat_template(AutoTokenizer.from_pretrained(MODEL))

In [ ]:
conv1 = [{"role": "user", "content": "What is 1+1 = 2?"},
        {"role": "assistant", "content": "<think> the user wants me to tell 1+1 which is 2</think>\nfinal_answer: 2"}]

conv2 = [{"role": "user", "content": "What is 1+1 = 2?"},
        {"role": "assistant", "content": "<think> the user wants me to tell 1+1 which is 2"},
        {"role": "assistant", "content": "</think>\nfinal_answer: 2"}]

In [ ]:
from transformers import AutoTokenizer
MODEL = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

def get_tokenizer_with_new_chat_template(tokenizer):
    to_delete = "{% if '</think>' in content %}{% set content = content.split('</think>')[-1] %}{% endif %}"
    new_template = tokenizer.get_chat_template().replace(to_delete, "")
    return AutoTokenizer.from_pretrained(MODEL, chat_template=new_template)

tokenizer = get_tokenizer_with_new_chat_template(AutoTokenizer.from_pretrained(MODEL))


conv1 = [{"role": "user", "content": "What is 1+1 = 2?"},
        {"role": "assistant", "content": "<think> the user wants me to tell 1+1 which is 2</think>\nfinal_answer: 2"}]

conv2 = [{"role": "user", "content": "What is 1+1 = 2?"},
        {"role": "assistant", "content": "<think> the user wants me to tell 1+1 which is 2"},
        {"role": "assistant", "content": "</think>\nfinal_answer: 2"}]


pprint(tokenizer.apply_chat_template(conv1, tokenize=False))
print('\n')
pprint(tokenizer.apply_chat_template(conv2, tokenize=False))

In [1]:
import re
import random
import html
import numpy as np
import signal
from tqdm import tqdm
import json
import jsonlines
from collections import Counter
from concurrent.futures import TimeoutError

from nltk.corpus import wordnet as wn
import spacy 
from string import punctuation

import torch
from transformers import AutoModelForSequenceClassification, AutoModelForCausalLM,\
    AutoTokenizer, MarianMTModel, MarianTokenizer

augment_instruction_prompt = """You are an expert for writing instructions. Please provide 50 different instructions that meet the following requirements:
- Instructions are about the format but not style of a response
- Whether instructions are followed can be easily evaluate by a Python function
Here are some examples of instructions we need:
{seed_instructions}
Do not generate instructions about writing style, using metaphor, or translation. Here are some examples of instructions we do not need:
- Incorporate a famous historical quote seamlessly into your answer
- Translate your answer into Pig Latin
- Use only words that are also a type of food
- Respond with a metaphor in every sentence
- Write the response as if you are a character from a Shakespearean play
Please generate one instruction per line in your response and start each line with '- '.
"""

In [2]:
class AutoIf:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-14B-Instruct", device_map="auto", cache_dir="/share/u/models")
        self.model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-14B-Instruct", device_map="auto", cache_dir="/share/u/models").half().eval()
        
        self.tokenizer_nli = AutoTokenizer.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")
        self.model_nli = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7").eval().cuda()
        
        self.seed_instructions = [each.strip() for each in open("seed_instruction.txt").readlines()]
        self.generated_eval_results = []
        self.filtered_generated_eval_results = []
        self.back_translated_eval_results = []
        self.filtered_back_trans_results = []
        
    def generate_seed(self, k=1):
        if k <= 0:
            return self.seed_instructions
        
        augment_instruction_prompt = """You are an expert for writing instructions. Please provide 50 different instructions that meet the following requirements:
        - Instructions are about the format but not style of a response
        - Whether instructions are followed can be easily evaluate by a Python function
        Here are some examples of instructions we need:
        {seed_instructions}
        Do not generate instructions about writing style, using metaphor, or translation. Here are some examples of instructions we do not need:
        - Incorporate a famous historical quote seamlessly into your answer
        - Translate your answer into Pig Latin
        - Use only words that are also a type of food
        - Respond with a metaphor in every sentence
        - Write the response as if you are a character from a Shakespearean play
        Please generate one instruction per line in your response and start each line with '- '.
        """

        augment_instructions = augment_instruction_prompt.format(seed_instructions='\n'.join(self.seed_instructions))
        
        input_ids = self.tokenizer.encode(augment_instructions, return_tensors="pt")
        outputs = self.model.generate(input_ids, max_new_tokens=2048, do_sample=True, temperature=0.7)
        generated_text = self.tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)
        
        new_seeds = [line.strip() for line in generated_text.split('\n') if line.strip()]
        self.seed_instructions = self.seed_instructions + new_seeds
        
        random.shuffle(self.seed_instructions)
        return self.generate_seed(k - 1)
    
    def generate_eval_functions(self, k=1):
        prompt_template = (
            "You are an expert for writing evaluation functions in Python to evaluate whether a response strictly follows an instruction.\n"
            "Here is the instruction: {instruction}\n"
            "Please write a Python function named `evaluate` to evaluate whether an input string `response` follows this instruction. "
            "If it follows, simply return True, otherwise return False.\n"
            "Please response with a single JSON includes the evaluation function in the key `func`, and a list of three test cases in the key `cases`, "
            "which includes an input in the key `input` and an expected output in the key `output` in (true, false).\n"
            "Here is an example of output JSON format: {{\"func\": JSON_STR(use only \\n instead of \n), \"cases\": [{{\"input\": str, \"output\": str}}]}}."
        )

        for instruction in self.seed_instructions:
            prompt = prompt_template.format(instruction=instruction)
            for _ in range(k):
                input_ids = self.tokenizer.encode(prompt, return_tensors="pt")
                outputs = self.model.generate(input_ids, max_new_tokens=2048, do_sample=True, temperature=0.7)
                generated_text = self.tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)
                self.generated_eval_results.append({
                    "prompt": prompt,
                    "instruction": instruction,
                    "gpt-answer": generated_text
                })
                
    def filter_generated_eval_results(self):
        collect_packages = []
        for result in self.generated_eval_results:
            res = result['gpt-answer']
            eval_funcs, test_cases = [], []
            for each in res:
                try:
                    json_dict = re.findall(r'```json(.*?)```', each, re.DOTALL)[0].strip()
                except IndexError:
                    continue
                try:
                    res_dict = json.loads(json_dict)
                except json.JSONDecodeError:
                    continue
                func = res_dict['func']
                if '\\n' in func:
                    func = func.replace('\\n', '\n')
                try:
                    exec(func)
                except Exception:
                    continue
                for line in func.split('\n'):
                    if 'import' in line or 'download' in line or 'requests' in line:
                        collect_packages.append(line)
        print(list(set(collect_packages)))

        def timeout_handler(signum, frame):
            raise TimeoutError("Function execution timed out")

        for result in tqdm(self.generated_eval_results):
            res = result['gpt-answer']
            eval_funcs, test_cases = [], []
            for each in tqdm(res):
                try:
                    json_dict = re.findall(r'```json(.*?)```', each, re.DOTALL)[0].strip()
                except IndexError:
                    continue
                try:
                    res_dict = json.loads(json_dict)
                except json.JSONDecodeError:
                    continue

                # func rejection and cleaning
                func = res_dict['func'].strip()
                func = '\n'.join([line for line in func.split('\n') if 'download' not in line and 'requests' not in line])
                try:
                    exec(func)
                except Exception:
                    continue
                eval_funcs.append(func)

                for each_case in res_dict['cases']:
                    try:
                        test_cases.append((each_case['input'], each_case['output']))
                    except KeyError:
                        print(each_case)

            eval_funcs = list(set(eval_funcs))
            test_cases = list(map(json.loads, set(map(json.dumps, test_cases))))
            if len(eval_funcs) < 3 or len(test_cases) < 10:
                continue

            filtered_test_cases = []
            for each in tqdm(test_cases):
                flag = False
                for func in eval_funcs:
                    local_vars = {}
                    try:
                        exec(func, globals(), local_vars)
                    except Exception:
                        continue
                    if 'evaluate' not in local_vars:
                        continue
                    eval_func = local_vars['evaluate']
                    try:
                        signal.signal(signal.SIGALRM, timeout_handler)
                        signal.alarm(5)
                        res_val = eval_func(each[0])
                    except Exception:
                        res_val = None
                    finally:
                        signal.alarm(0)
                    if res_val is not None and res_val == each[1]:
                        flag = True
                if flag:
                    filtered_test_cases.append(each)

            scored_funcs = []
            for func in tqdm(eval_funcs):
                local_vars = {}
                try:
                    exec(func, globals(), local_vars)
                except Exception:
                    continue
                if 'evaluate' not in local_vars:
                    continue
                eval_func = local_vars['evaluate']
                acc = []
                for inp, out in filtered_test_cases:
                    try:
                        signal.signal(signal.SIGALRM, timeout_handler)
                        signal.alarm(5)
                        res_val = eval_func(inp)
                    except Exception:
                        res_val = None
                    finally:
                        signal.alarm(0)
                    if res_val is None or res_val != out:
                        acc.append(0)
                    else:
                        acc.append(1)
                acc = np.mean(acc) if acc else 0
                scored_funcs.append((func, acc))
            valid_funcs = [each for each in scored_funcs if each[1] >= 0.8]
            if not valid_funcs:
                continue

            self.filtered_generated_eval_results.append({
                "instruction": result['instruction'],
                "eval_func": valid_funcs,
                "cases": filtered_test_cases
            })
            

    def generate_back_trans_instructions(self):
        self.back_translated_eval_results = []
        count = 0
        filter_count = 0

        for i, line in enumerate(self.filtered_generated_eval_results):
            funcs = line["eval_func"][:3]

            instruction_prompt = f"""You are an expert in converting the Python eval function code into the corresponding instruction text. I will provide the eval function code. Please strictly follow the code to convert it into the corresponding instruction text. Here's an example: 

[["def evaluate(response):\n    return 'e' not in response.lower()", 1.0], ["def evaluate(response):\n    words = response.split()\n    for word in response.split():\n        if 'e' in word.lower():\n            return False\n    return True", 1.0], ["def evaluate(response):\n    return all('e' not in word.lower() for word in response.split())", 1.0]] 

["Answer without using any words that contain the letter 'E'.","Answer with words that do not contain the letter 'E'.","Answer with words that do not contain the letter 'E'."] Please convert the following eval function into instructions stored in a list: 

{funcs}"""

            input_ids = self.tokenizer.encode(instruction_prompt, return_tensors="pt").cuda()
            outputs = self.model.generate(input_ids, max_new_tokens=1024, do_sample=True, temperature=0.7)
            generated_text = self.tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)

            try:
                back_instruction = json.loads(generated_text)
            except Exception:
                filter_count += 1
                continue

            line["back_instruction"] = back_instruction
            self.back_translated_eval_results.append(line)
            count += 1
            

    def filter_back_trans_instructions(self):
        count = 0 
        filter_count = 0

        for i, line in enumerate(self.back_translated_eval_results):
            back_instructions = line["back_instruction"]
            ori_ins = line["instruction"]

            nli_scores = []
            for back_ins in back_instructions[:3]:
                premise = ori_ins
                hypothesis = back_ins

                inputs = self.tokenizer_nli(premise, hypothesis, truncation=True, return_tensors="pt").cuda()
                output = self.model_nli(inputs["input_ids"].cuda())
                prediction = torch.softmax(output["logits"][0], -1).tolist()
                label_names = ["entailment", "neutral", "contradiction"]
                prediction_dict = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
                max_label = max(prediction_dict, key=prediction_dict.get)
                nli_scores.append(max_label)

            line["nli_scores"] = nli_scores
            if "contradiction" in nli_scores:
                filter_count += 1
                continue
            else:
                self.filtered_back_trans_results.append(line)
            count += 1

In [ ]:
autoif = AutoIf()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
with open("", "r") as file:
    data = [json.loads(line) for line in file]
    
autoif.filter_generated_eval_results

In [15]:
model, tokenizer = autoif.model.half(), autoif.tokenizer

In [20]:
# autoif.generate_seed()
seed_instructions = [each.strip() for each in open("seed_instruction.txt").readlines()]
augment_instruction_prompt = """You are an expert for writing instructions. Please provide 50 different instructions that meet the following requirements:
- Instructions are about the format but not style of a response
- Whether instructions are followed can be easily evaluate by a Python function
Here are some examples of instructions we need:
{seed_instructions}
Do not generate instructions about writing style, using metaphor, or translation. Here are some examples of instructions we do not need:
- Incorporate a famous historical quote seamlessly into your answer
- Translate your answer into Pig Latin
- Use only words that are also a type of food
- Respond with a metaphor in every sentence
- Write the response as if you are a character from a Shakespearean play
Please generate one instruction per line in your response and start each line with '- '.
"""

augment_instructions = augment_instruction_prompt.format(seed_instructions='\n'.join(seed_instructions))

input_ids = tokenizer.encode(augment_instructions, return_tensors="pt").cuda()
outputs = model.generate(input_ids, max_new_tokens=1024, do_sample=False, temperature=0.7)
generated_text = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)

new_seeds = [line.strip() for line in generated_text.split('\n') if line.strip()]
seed_instructions = seed_instructions + new_seeds

random.shuffle(seed_instructions)

/share/u/harshraj/.conda/envs/r1/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/share/u/harshraj/.conda/envs/r1/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/share/u/harshraj/.conda/envs/r1/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


In [21]:
seed_instructions

['- Respond with a list of bullet points',
 '- Use only questions to form your reply',
 '- Answer with a single sentence that is exactly 100 words long',
 '- Use no adjectives or adverbs',
 "- Use no words containing the letter 'E'",
 '- Use only words that rhyme with "land"',
 '- Use only uppercase letters',
 '- Use only words that rhyme with "tree"',
 '- Use only words that have an X in them',
 '- Answer with',
 '- Respond with a haiku (5-7-5 syllable structure)',
 "- Use words that end with '-ing'",
 '- Use only words that rhyme with "cat"',
 "- Use words that end with '-ing'",
 'Write the response as a limerick',
 'Use only words that are colors',
 '- Respond with a list of bullet points',
 '- Use only words that rhyme with "dog"',
 '- Use only words with double letters (e.g., "bookkeeper")',
 '- Write the response as a limerick',
 '- Write the response in iambic pentameter',
 "- Answer with words that begin with the letter 'B'",
 'Use only words that start and end with the same le

In [23]:
seed_instructions = seed_instructions[:2]

In [24]:
# autoif.generate_eval_functions()
generated_eval_results = []
prompt_template = (
    "You are an expert for writing evaluation functions in Python to evaluate whether a response strictly follows an instruction.\n"
    "Here is the instruction: {instruction}\n"
    "Please write a Python function named `evaluate` to evaluate whether an input string `response` follows this instruction. "
    "If it follows, simply return True, otherwise return False.\n"
    "Please response with a single JSON includes the evaluation function in the key `func`, and a list of three test cases in the key `cases`, "
    "which includes an input in the key `input` and an expected output in the key `output` in (true, false).\n"
    "Here is an example of output JSON format: {{\"func\": JSON_STR(use only \\n instead of \n), \"cases\": [{{\"input\": str, \"output\": str}}]}}."
)

for instruction in seed_instructions:
    prompt = prompt_template.format(instruction=instruction)
    for _ in range(1):
        print("prompt:", prompt)
        input_ids = tokenizer.encode(prompt, return_tensors="pt").cuda()
        print("input_ids:", input_ids)
        outputs = model.generate(input_ids, max_new_tokens=1024, do_sample=False, temperature=0.7)
        generated_text = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)
        generated_eval_results.append({
            "prompt": prompt,
            "instruction": instruction,
            "gpt-answer": generated_text
        })

prompt: You are an expert for writing evaluation functions in Python to evaluate whether a response strictly follows an instruction.
Here is the instruction: - Respond with a list of bullet points
Please write a Python function named `evaluate` to evaluate whether an input string `response` follows this instruction. If it follows, simply return True, otherwise return False.
Please response with a single JSON includes the evaluation function in the key `func`, and a list of three test cases in the key `cases`, which includes an input in the key `input` and an expected output in the key `output` in (true, false).
Here is an example of output JSON format: {"func": JSON_STR(use only \n instead of 
), "cases": [{"input": str, "output": str}]}.
input_ids: tensor([[ 2610,   525,   458,  6203,   369,  4378, 16460,  5746,   304, 13027,
           311, 15442,  3425,   264,  2033, 25470, 11017,   458,  7600,   624,
          8420,   374,   279,  7600,    25,   481, 39533,   448,   264,  1140,
   

/share/u/harshraj/.conda/envs/r1/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/share/u/harshraj/.conda/envs/r1/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/share/u/harshraj/.conda/envs/r1/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


prompt: You are an expert for writing evaluation functions in Python to evaluate whether a response strictly follows an instruction.
Here is the instruction: - Use only questions to form your reply
Please write a Python function named `evaluate` to evaluate whether an input string `response` follows this instruction. If it follows, simply return True, otherwise return False.
Please response with a single JSON includes the evaluation function in the key `func`, and a list of three test cases in the key `cases`, which includes an input in the key `input` and an expected output in the key `output` in (true, false).
Here is an example of output JSON format: {"func": JSON_STR(use only \n instead of 
), "cases": [{"input": str, "output": str}]}.
input_ids: tensor([[ 2610,   525,   458,  6203,   369,  4378, 16460,  5746,   304, 13027,
           311, 15442,  3425,   264,  2033, 25470, 11017,   458,  7600,   624,
          8420,   374,   279,  7600,    25,   481,  5443,  1172,  4755,   311,
  

In [25]:
generated_eval_results

[{'prompt': 'You are an expert for writing evaluation functions in Python to evaluate whether a response strictly follows an instruction.\nHere is the instruction: - Respond with a list of bullet points\nPlease write a Python function named `evaluate` to evaluate whether an input string `response` follows this instruction. If it follows, simply return True, otherwise return False.\nPlease response with a single JSON includes the evaluation function in the key `func`, and a list of three test cases in the key `cases`, which includes an input in the key `input` and an expected output in the key `output` in (true, false).\nHere is an example of output JSON format: {"func": JSON_STR(use only \\n instead of \n), "cases": [{"input": str, "output": str}]}.',
  'instruction': '- Respond with a list of bullet points',
  'gpt-answer': ' The function should be defined as `def evaluate(response):`. The function should be self-contained and import any necessary packages within it.\n```json\n{\n  "f

In [ ]:
collect_packages = []
for result in generated_eval_results:
    res = result['gpt-answer']
    eval_funcs, test_cases = [], []
    for each in res:
        try:
            json_dict = re.findall(r'```json(.*?)```', each, re.DOTALL)[0].strip()
        except IndexError:
            continue
        try:
            res_dict = json.loads(json_dict)
        except json.JSONDecodeError:
            continue
        func = res_dict['func']
        if '\\n' in func:
            func = func.replace('\\n', '\n')
        try:
            exec(func)
        except Exception:
            continue
        for line in func.split('\n'):
            if 'import' in line or 'download' in line or 'requests' in line:
                collect_packages.append(line)
print(list(set(collect_packages)))

def timeout_handler(signum, frame):
    raise TimeoutError("Function execution timed out")

for result in tqdm(generated_eval_results):
    res = result['gpt-answer']
    eval_funcs, test_cases = [], []
    for each in tqdm(res):
        try:
            json_dict = re.findall(r'```json(.*?)```', each, re.DOTALL)[0].strip()
        except IndexError:
            continue
        try:
            res_dict = json.loads(json_dict)
        except json.JSONDecodeError:
            continue

        # func rejection and cleaning
        func = res_dict['func'].strip()
        func = '\n'.join([line for line in func.split('\n') if 'download' not in line and 'requests' not in line])
        try:
            exec(func)
        except Exception:
            continue
        eval_funcs.append(func)

        for each_case in res_dict['cases']:
            try:
                test_cases.append((each_case['input'], each_case['output']))
            except KeyError:
                print(each_case)

    eval_funcs = list(set(eval_funcs))
    test_cases = list(map(json.loads, set(map(json.dumps, test_cases))))
    if len(eval_funcs) < 3 or len(test_cases) < 10:
        continue

    filtered_test_cases = []
    for each in tqdm(test_cases):
        flag = False
        for func in eval_funcs:
            local_vars = {}
            try:
                exec(func, globals(), local_vars)
            except Exception:
                continue
            if 'evaluate' not in local_vars:
                continue
            eval_func = local_vars['evaluate']
            try:
                signal.signal(signal.SIGALRM, timeout_handler)
                signal.alarm(5)
                res_val = eval_func(each[0])
            except Exception:
                res_val = None
            finally:
                signal.alarm(0)
            if res_val is not None and res_val == each[1]:
                flag = True
        if flag:
            filtered_test_cases.append(each)

    scored_funcs = []
    for func in tqdm(eval_funcs):
        local_vars = {}
        try:
            exec(func, globals(), local_vars)
        except Exception:
            continue
        if 'evaluate' not in local_vars:
            continue
        eval_func = local_vars['evaluate']
        acc = []
        for inp, out in filtered_test_cases:
            try:
                signal.signal(signal.SIGALRM, timeout_handler)
                signal.alarm(5)
                res_val = eval_func(inp)
            except Exception:
                res_val = None
            finally:
                signal.alarm(0)
            if res_val is None or res_val != out:
                acc.append(0)
            else:
                acc.append(1)
        acc = np.mean(acc) if acc else 0
        scored_funcs.append((func, acc))
    valid_funcs = [each for each in scored_funcs if each[1] >= 0.8]
    if not valid_funcs:
        continue

    self.filtered_generated_eval_results.append({
        "instruction": result['instruction'],
        "eval_func": valid_funcs,
        "cases": filtered_test_cases
    })


In [ ]:
import re
import random
import html
import numpy as np
import signal
from tqdm import tqdm
import json
import jsonlines
from collections import Counter
from concurrent.futures import TimeoutError

from nltk.corpus import wordnet as wn
import spacy 
from string import punctuation

import torch
from transformers import AutoModelForSequenceClassification, AutoModelForCausalLM,\
    AutoTokenizer, MarianMTModel, MarianTokenizer

In [ ]:

class AutoIf:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-72B-Instruct")
        self.model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-72B-Instruct", device_map="auto").half().eval()
        
        self.tokenizer_nli = AutoTokenizer.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")
        self.model_nli = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7").eval().cuda()
        
        self.seed_instructions = [each.strip() for each in open("/share/u/harshraj/CotIF/notebooks/seed_instruction.txt").readlines()]
        self.generated_eval_functions = []
        self.filtered_generated_eval_functions = []
        self.generated_instructions = []
        self.filtered_generated_instructions = []
        self.generated_responses = []
        self.filtered_generated_responses = []
        self.filtered2_generated_responses = []
        
    def compile(self, text):
        self.generate_seed()
        self.generate_eval_function()
        self.filter_generated_eval_function()
        self.generate_instruction()
        self.filter_generated_instruction()
        self.generate_response(text)
        self.filter_generated_response()
        self.filter2_generated_response()
        return self.filtered2_generated_responses
        
    def generate_seed(self, k=2):
        if k <= 0:
            return self.seed_instructions
        
        augment_instruction_prompt = """You are an expert for writing instructions. Please provide 50 different instructions that meet the following requirements:
        - Instructions are about the format but not style of a response
        - Whether instructions are followed can be easily evaluate by a Python function
        Here are some examples of instructions we need:
        {seed_instructions}
        Do not generate instructions about writing style, using metaphor, or translation. Here are some examples of instructions we do not need:
        - Incorporate a famous historical quote seamlessly into your answer
        - Translate your answer into Pig Latin
        - Use only words that are also a type of food
        - Respond with a metaphor in every sentence
        - Write the response as if you are a character from a Shakespearean play
        Please generate one instruction per line in your response and start each line with '- '.
        """

        augment_instructions = augment_instruction_prompt.format(seed_instructions='\n'.join(self.seed_instructions))
        
        input_ids = self.tokenizer.encode(augment_instructions, return_tensors="pt").cuda()
        outputs = self.model.generate(input_ids, max_length=1024, do_sample=True, temperature=0.7)
        generated_text = self.tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)
        
        new_seeds = [line.strip() for line in generated_text.split('\n') if line.strip()]
        self.seed_instructions = self.seed_instructions + new_seeds
        
        random.shuffle(self.seed_instructions)
        return self.generate_seed(k - 1)
    
    def generate_eval_function(self, k=2):
        prompt_template = (
            "You are an expert for writing evaluation functions in Python to evaluate whether a response strictly follows an instruction.\n"
            "Here is the instruction: {instruction}\n"
            "Please write a Python function named `evaluate` to evaluate whether an input string `response` follows this instruction. "
            "If it follows, simply return True, otherwise return False.\n"
            "Please response with a single JSON includes the evaluation function in the key `func`, and a list of three test cases in the key `cases`, "
            "which includes an input in the key `input` and an expected output in the key `output` in (true, false).\n"
            "Here is an example of output JSON format: {{\"func\": JSON_STR(use only \\n instead of \n), \"cases\": [{{\"input\": str, \"output\": str}}]}}."
        )

        for instruction in self.seed_instructions:
            prompt = prompt_template.format(instruction=instruction)
            for _ in range(k):
                input_ids = self.tokenizer.encode(prompt, return_tensors="pt").cuda()
                outputs = self.model.generate(input_ids, max_length=1024, do_sample=True, temperature=0.7)
                generated_text = self.tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)
                self.generated_eval_functions.append({
                    "prompt": prompt,
                    "instruction": instruction,
                    "gpt-answer": generated_text
                })
                
    def filter_generated_eval_function(self):
        collect_packages = []
        for result in self.generated_eval_functions:
            res = result['gpt-answer']
            eval_funcs, test_cases = [], []
            for each in res:
                try:
                    json_dict = re.findall(r'```json(.*?)```', each, re.DOTALL)[0].strip()
                except IndexError:
                    continue
                try:
                    res_dict = json.loads(json_dict)
                except json.JSONDecodeError:
                    continue
                func = res_dict['func']
                if '\\n' in func:
                    func = func.replace('\\n', '\n')
                try:
                    exec(func)
                except Exception:
                    continue
                for line in func.split('\n'):
                    if 'import' in line or 'download' in line or 'requests' in line:
                        collect_packages.append(line)
        print(list(set(collect_packages)))

        for result in tqdm(self.generated_eval_functions):
            res = result['gpt-answer']
            eval_funcs, test_cases = [], []
            for each in tqdm(res):
                try:
                    json_dict = re.findall(r'```json(.*?)```', each, re.DOTALL)[0].strip()
                except IndexError:
                    continue
                try:
                    res_dict = json.loads(json_dict)
                except json.JSONDecodeError:
                    continue

                # func rejection and cleaning
                func = res_dict['func'].strip()
                func = '\n'.join([line for line in func.split('\n') if 'download' not in line and 'requests' not in line])
                try:
                    exec(func)
                except Exception:
                    continue
                eval_funcs.append(func)

                for each_case in res_dict['cases']:
                    try:
                        test_cases.append((each_case['input'], each_case['output']))
                    except KeyError:
                        print(each_case)

            eval_funcs = list(set(eval_funcs))
            test_cases = list(map(json.loads, set(map(json.dumps, test_cases))))
            if len(eval_funcs) < 3 or len(test_cases) < 10:
                continue

            filtered_test_cases = []
            for each in tqdm(test_cases):
                flag = False
                for func in eval_funcs:
                    local_vars = {}
                    try:
                        exec(func, globals(), local_vars)
                    except Exception:
                        continue
                    if 'evaluate' not in local_vars:
                        continue
                    eval_func = local_vars['evaluate']
                    try:
                        signal.signal(signal.SIGALRM, timeout_handler)
                        signal.alarm(5)
                        res_val = eval_func(each[0])
                    except Exception:
                        res_val = None
                    finally:
                        signal.alarm(0)
                    if res_val is not None and res_val == each[1]:
                        flag = True
                if flag:
                    filtered_test_cases.append(each)

            scored_funcs = []
            for func in tqdm(eval_funcs):
                local_vars = {}
                try:
                    exec(func, globals(), local_vars)
                except Exception:
                    continue
                if 'evaluate' not in local_vars:
                    continue
                eval_func = local_vars['evaluate']
                acc = []
                for inp, out in filtered_test_cases:
                    try:
                        signal.signal(signal.SIGALRM, timeout_handler)
                        signal.alarm(5)
                        res_val = eval_func(inp)
                    except Exception:
                        res_val = None
                    finally:
                        signal.alarm(0)
                    if res_val is None or res_val != out:
                        acc.append(0)
                    else:
                        acc.append(1)
                acc = np.mean(acc) if acc else 0
                scored_funcs.append((func, acc))
            valid_funcs = [each for each in scored_funcs if each[1] >= 0.8]
            if not valid_funcs:
                continue

            self.filtered_generated_eval_functions.append({
                "instruction": result['instruction'],
                "eval_func": valid_funcs,
                "cases": filtered_test_cases
            })
            

    def generate_instruction(self, k=2):
        count = 0
        filter_count = 0

        for line in tqdm(self.filtered_generated_eval_functions, desc="Generating back-translated instructions"):
            funcs = line["eval_func"][:3]

            instruction_prompt = f"""You are an expert in converting the Python eval function code into the corresponding instruction text. I will provide the eval function code. Please strictly follow the code to convert it into the corresponding instruction text. Here's an example: 

[["def evaluate(response):\n    return 'e' not in response.lower()", 1.0], ["def evaluate(response):\n    words = response.split()\n    for word in response.split():\n        if 'e' in word.lower():\n            return False\n    return True", 1.0], ["def evaluate(response):\n    return all('e' not in word.lower() for word in response.split())", 1.0]] 

["Answer without using any words that contain the letter 'E'.","Answer with words that do not contain the letter 'E'.","Answer with words that do not contain the letter 'E'."] Please convert the following eval function into instructions stored in a list: 

{funcs}"""

            for _ in range(k):
                input_ids = self.tokenizer.encode(instruction_prompt, return_tensors="pt").cuda()
                outputs = self.model.generate(input_ids, max_length=1024, do_sample=True, temperature=0.7)
                generated_text = self.tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)

                try:
                    back_instruction = json.loads(generated_text)
                    break
                except Exception:
                    filter_count += 1
                    continue

            line["back_instruction"] = back_instruction
            self.generated_instructions.append(line)
            count += 1
            

    def filter_generated_instruction(self):
        count = 0 
        filter_count = 0

        for line in tqdm(self.generated_instructions, desc="Filtering back-translated instructions"):
            back_instructions = line["back_instruction"]
            ori_ins = line["instruction"]

            nli_scores = []
            for back_ins in back_instructions[:3]:
                premise = ori_ins
                hypothesis = back_ins

                inputs = self.tokenizer_nli(premise, hypothesis, truncation=True, return_tensors="pt")
                output = self.model_nli(inputs["input_ids"].cuda())
                prediction = torch.softmax(output["logits"][0], -1).tolist()
                label_names = ["entailment", "neutral", "contradiction"]
                prediction_dict = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
                max_label = max(prediction_dict, key=prediction_dict.get)
                nli_scores.append(max_label)

            line["nli_scores"] = nli_scores
            if "contradiction" in nli_scores:
                filter_count += 1
                continue
            else:
                self.filtered_generated_instructions.append(line)
            count += 1
            
    def generate_response(self, text, k=2):
        for instruction in self.filtered_generated_instructions:
            prompt = (
                f"Please answer the query strictly following the instruction.\n"
                f"[instruction] {instruction['instruction']}\n"
                f"[Query] {text}"
            )
            
            responses = []
            for _ in range(k):
                input_ids = self.tokenizer.encode(prompt, return_tensors="pt").cuda()
                outputs = self.model.generate(input_ids, max_length=1024, do_sample=True, temperature=0.7)
                generated_text = self.tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)
                responses.append(generated_text)
            
            self.generated_responses.append({
                "instruction": instruction['instruction'],
                "prompt": prompt,
                "gpt-answer": responses,
                "eval_func": instruction["eval_func"],
            })
                
    def filter_generated_response(self):
        filtered_samples = []
        for result in tqdm(self.generated_responses, desc="Filtering back translated responses"):
            eval_funcs = []
            for func, score in result['eval_func']:
                local_vars = {}
                try:
                    exec(func, globals(), local_vars)
                except Exception as e:
                    print("Error executing eval function:", e)
                    continue
                if 'evaluate' in local_vars:
                    eval_funcs.append(local_vars['evaluate'])
            
            filter_responses = []
            for response in result['gpt-answer']:
                acc = []
                for eval_func in eval_funcs:
                    try:
                        signal.signal(signal.SIGALRM, timeout_handler)
                        signal.alarm(5)
                        res = eval_func(response)
                    except Exception as e:
                        print(e)
                        res = None
                    finally:
                        signal.alarm(0)
                    if res is not None:
                        try:
                            acc.append(int(res))
                        except Exception:
                            continue
                acc = np.mean(acc) if acc else 0
                if acc > 0:
                    filter_responses.append(response)
            
            for each in filter_responses:
                try:
                    query_match = re.findall(r'\[Query\](.*)$', result['prompt'], re.DOTALL)
                    query = query_match[0].strip() if query_match else ""
                    filtered_samples.append({
                        'instruction': result['instruction'],
                        'query': query,
                        'response': each
                    })
                except IndexError:
                    print("Prompt extraction error:", result['prompt'])
        
        self.filtered_generated_responses = list(map(json.loads, set(map(json.dumps, filtered_samples))))
        
    def filter2_generated_response(self, k=2): 
        prompt_template = (
            "You are an expert that is good at judging whether a response is following the instruction and query.\n"
            "[Instruction] {instruction}\n"
            "[Query] {query}\n"
            "[Response] {response}\n"
            "Please notice that the response may not be helpful as it needs to strictly follow the requirements in the Instruction.\n"
            "You need to judge whether the response answers the query. Please first provide a detailed analysis and then give a score ranking from 0 to 10 at the last line.\n"
            "Scoring 0 means the response is totally unrelated to the query, while scoring 10 means the response is helpful and highly related to the query.\n"
            "Please only provide a score in the format `Score: {{score}}` without any other contents at the last line."
        )
        for each in self.filtered_generated_responses:
            each['prompt'] = prompt_template.format(
                instruction=each['instruction'],
                query=each['query'],
                response=each['response']
            )
            each['gen'] = []
            for _ in range(k):
                input_ids = self.tokenizer.encode(each['prompt'], return_tensors="pt").cuda()
                outputs = self.model.generate(input_ids, max_length=1024, do_sample=True, temperature=0.7)
                generated_text = self.tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)
                each['gen'].append(generated_text)
            
            scores = []
            for each in each['gen']:
                score = re.findall(r'Score: (\d+?)$', each)
                if score:
                    scores.append(int(score[0]))
            score = np.mean(scores) if scores else 0
            if score > 8: # quality score
                self.filtered2_generated_responses.append(each)

In [ ]:
autoif = AutoIf()

NameError: name 'AutoTokenizer' is not defined